In [1]:
# https://towardsdatascience.com/hands-on-generative-adversarial-networks-gan-for-signal-processing-with-python-ff5b8d78bd28

In [2]:
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
import numpy as np
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM
from matplotlib import pyplot
import matplotlib.pyplot as plt

In [3]:
# prepare real data
%run ./read_file.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d02c02f2-df40-47db-af74-9e5fbe5eee69;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
downloading https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar ...
	[SUCCESSFUL ] graphframes#graphframes;0.8.2-spark3.2-s_2.12!graphframes.jar (34ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.16/slf4j-api-1.7.16.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.16!slf4j-api.jar (27ms)
:: resolution report :: resolve 659ms :: artifacts dl 65ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	--------

23/04/14 14:13:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/14 14:13:19 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [4]:
# https://stackoverflow.com/questions/68036975/valueerror-shape-must-be-at-least-rank-3-but-is-rank-2-for-node-biasadd
# config for rank error in lstm
tf.keras.backend.set_image_data_format("channels_last")

In [5]:
def prepare_real_samples(ds, lc=300):
    # Calculate the number of malicious and non-malicious records
    malicious_ds = ds.filter(col("malicious") == 1).limit(lc)
    ds_events = malicious_ds.select('event_sequence').rdd.flatMap(lambda x: x).collect()
    train_sequences = tokenizer.texts_to_sequences(ds_events)
    train_padded = tf.keras.utils.pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    train_ds = tf.reshape(train_padded, (-1, len(train_padded), 6))
    train_labels_ds = ones((1, input_len, 6, 1, 1))
    return train_ds, train_labels_ds

In [6]:
data = spark.read.parquet(*["s3a://sapient-bucket-trusted/prod/graph/encoded/real/23Sep3/*"]).cache()

In [7]:
# Set Config
max_length = 6
sequence_length = 6
max_features = 10000
padding_type = 'post'
trunc_type = 'post'
training_portion = 0.8
# number of samples to make
input_len = 10

In [8]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
ds_all_events = data.select('event_sequence').rdd.flatMap(lambda x: x).collect()
tokenizer.fit_on_texts(ds_all_events)

In [9]:
x_real, y_real = prepare_real_samples(data, lc=input_len)

In [69]:
word_index = tokenizer.word_index
vocab_count = len(word_index)
embedding_dim = len(word_index)

In [11]:
batch_size = 32

In [12]:
vocab_count

111

In [13]:
METRICS = [
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalseNegatives(name='fn'), 
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc'),
        keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [14]:
def rescale_value(val, old_range, new_range):
    """
    this converts a value within a range of values to an integer within another range 
    """
    new_val = tf.cast((val - old_range[0]) / (old_range[1] - old_range[0]) * (new_range[1] - new_range[0]) + new_range[0],tf.int32)
    return new_val

In [15]:
# define the standalone generator model
# https://www.tensorflow.org/tutorials/generative/dcgan#the_generator
def define_generator():
    model = tf.keras.Sequential([
            tf.keras.layers.Embedding(input_dim=vocab_count, output_dim=vocab_count, input_length=sequence_length), # outputs shape (input_len, sequence_length, embedding_dim)
            tf.keras.layers.LSTM(64, return_sequences=True), # outputs shape (input_len, sequence_length, embedding_dim)
            tf.keras.layers.Dense(1, activation='sigmoid'), # outputs shape (input_len, sequence_length, embedding_dim)
    ])
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=METRICS)
    return model

In [16]:
# define the standalone discriminator model
def define_discriminator():
    model = tf.keras.Sequential([
        # tf.keras.layers.Input(shape=(input_len, 6)),
        tf.keras.layers.Flatten(input_shape=(6,)), # Flatten the input tensor
        # tf.keras.layers.Embedding(100 + 1, 16),
        tf.keras.layers.Dense(6, activation='tanh'), # Add an additional Dense layer to get desired output shape
        # tf.keras.layers.Reshape((input_len, 6)), # Reshape output to desired shape
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=METRICS)
    return model

In [17]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
    # connect them
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=METRICS)
    return model

In [41]:
def generate_latent_space():
    n = tf.random.uniform(shape=[input_len, 6], minval=1, maxval=vocab_count, dtype=tf.int32)
    return n

In [20]:
def generate_fake_samples(generator, latent_dim=3):
    # generate points in latent space
    n_ds = generate_latent_space()
    # predict outputs
    X = generator.predict(n_ds, verbose=0)
    # convert the samples using the vocab
    # create class labels
    y = np.zeros((X.shape[0], 6))
    return X, y

In [21]:
metrics = ["accuracy", "loss", "prc", "recall", "auc", "precision"]
def plot_graphs(history, metrics):
    fig, axs = plt.subplots(nrows = 2, ncols=3, figsize=(12, 6), layout="constrained")
    for ax, m in zip(axs.flat, metrics):
        ax.plot(history.history[m])
        ax.plot(history.history['val_'+ m])
        ax.set_title(f'{m}')

In [22]:
# use the generator to generate n fake examples, with class labels
def train(g_model, d_model, gan_model, latent_dim=3, n_epochs=10, n_batch=128, n_eval=20):
    # manually enumerate epochs
    for i in range(n_epochs):
        # prepare real samples
        # x_real, y_real = generate_real_samples(n_batch)
        # prepare fake examples using the generator
        x_fake, y_fake = generate_fake_samples(g_model, n_batch)
        # update discriminator
        d_model.train_on_batch(x_real, y_real)
        d_model.train_on_batch(x_fake, y_fake)
        # prepare points in latent space as input for the generator
        x_gan = generate_latent_space(n_batch)
        # create inverted labels for the fake samples
        y_gan = np.ones((n_batch, input_len, 6))
        # update the generator via the discriminator's error
        gan_model.train_on_batch(x_gan, y_gan)
        # evaluate the model every n_eval epochs
        if (i+1) % n_eval == 0:
            print('Number of epochs = %i'%(i+1))

In [102]:
g_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_count, output_dim=vocab_count, input_length=input_len), # outputs shape (input_len, sequence_length, embedding_dim)
    # tf.keras.layers.LSTM(vocab_count, return_sequences=True), # outputs shape (input_len, sequence_length, embedding_dim)
    ])
# Compile the model
g_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=METRICS)

In [103]:
ls = generate_latent_space()
ls.shape

TensorShape([10, 6])

In [104]:
predicted_embeddings = g_model.predict(ls, verbose=0)

ValueError: in user code:

    File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/keras/engine/training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/keras/engine/training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/keras/engine/training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/keras/engine/training.py", line 2079, in predict_step
        return self(x, training=False)
    File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
   

In [93]:
predicted_embeddings.shape

(10, 6, 1)

In [94]:
predicted_integers = predictions.argmax(axis=1)

In [95]:
predicted_integers

array([[1],
       [2],
       [2],
       [0],
       [0],
       [5],
       [0],
       [1],
       [1],
       [0]])

In [82]:
np.max(predicted_integers)

5

In [83]:
np.min(predicted_integers)

0

In [22]:
# Test combination of generator and discriminator
generator = define_generator()
discriminator = define_discriminator()
gan = define_gan(generator, discriminator)

In [25]:
x_a = generate_latent_space()

In [24]:
x_fake, y_fake = generate_fake_samples(generator, input_len)

ValueError: in user code:

    File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/keras/engine/training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/keras/engine/training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/keras/engine/training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/keras/engine/training.py", line 2079, in predict_step
        return self(x, training=False)
    File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
   

In [86]:
# test gan created data and model

In [87]:
x_gan = generate_latent_space()
x_gan = x_gan[np.newaxis, :]
y_gan = ones((1, input_len, 6))

In [88]:
x_gan.shape

TensorShape([1, 32, 10, 6])

In [89]:
y_gan.shape

(1, 10, 6)

In [91]:
discriminator.train_on_batch(x_gan, y_gan)

ValueError: in user code:

    File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File

In [40]:
# Create real gan

In [41]:
train(generator, discriminator, gan, n_epochs=20)

ValueError: in user code:

    File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File

In [ ]:
# create data based on the trained model
si = generate_latent_space()
sp = gan.predict(si)

In [ ]:
sp[0]

In [ ]:
np.min(sp)

In [ ]:
np.max(sp)

In [ ]:
predicted_indices = np.argmax(sp, axis=-1)

In [ ]:
#sp_ds = tf.reshape(sp, (input_len, 6, 1))

In [ ]:
np.max(si)

In [ ]:
np.max(sp_ds)

In [ ]:
np.min(sp_ds)

In [ ]:
print(sp_ds[20:])

In [ ]:
table = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(
        keys=vocab, values=tf.constant(list(range(vocab_count)),
                                       dtype=tf.int64)),
    default_value=0)

In [ ]:
X = np.zeros((1, sequence_length, len(vocab)))

In [ ]:
X.shape